In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from get_images import get_images

In [ ]:
!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

In [ ]:
!mkdir data_mnist
!mv *ubyte* data_mnist

In [ ]:
# MNIST path
mnist_path = './data_mnist/'
x_train_num, y_train_num, x_test_num, y_test_num = get_images(mnist_path)

x_train = x_train_num[:50000].reshape(50000, -1).astype(np.float32)
y_train = y_train_num[:50000].reshape(50000, 1)

x_val = x_train_num[50000:].reshape(10000, -1).astype(np.float)
y_val = y_train_num[50000:].reshape(10000, 1)

x_test = x_test_num.copy().reshape(10000, -1).astype(np.float)
y_test = y_test_num.copy().reshape(10000, 1)

In [ ]:
def normalise(x_mean, x_std, x_data):
    return (x_data - x_mean) / x_std

In [ ]:
x_mean = x_train.mean()
x_std = x_train.std()

x_train = normalise(x_mean, x_std, x_train)
x_val = normalise(x_mean, x_std, x_val)
x_test = normalise(x_mean, x_std, x_test)

In [ ]:
x_train.mean(), x_train.std()

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
def plot_number(image):
    plt.figure(figsize=(5,5))
    plt.imshow(image.squeeze(), cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.show()

In [ ]:
rnd_idx = np.random.randint(len(y_test))
print(f'La imagen muestreada representa un: {y_test[rnd_idx, 0]}')
plot_number(x_test_num[rnd_idx])

In [ ]:
def create_minibatches(x, y, mb_size, shuffle = True):
    '''
    x  #muestras, 784
    y #muestras, 1
    '''
    assert x.shape[0] == y.shape[0], 'Error en cantidad de muestras'
    total_data = x.shape[0]
    if shuffle: 
        idxs = np.arange(total_data)
        np.random.shuffle(idxs)
        x = x[idxs]
        y = y[idxs]  
    return ((x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size))

In [ ]:
x_train_tensor = torch.tensor(x_train.copy())
y_train_tensor = torch.tensor(y_train.copy())

x_val_tensor = torch.tensor(x_val.copy())
y_val_tensor = torch.tensor(y_val.copy())

x_test_tensor = torch.tensor(x_test.copy())
y_test_tensor = torch.tensor(y_test.copy())

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Estammos usando: {device}')

In [ ]:
def accuracy(model, x, y, mb_size):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for (xi, yi) in create_minibatches(x, y, mb_size):
            xi = xi.to(device=device, dtype = torch.float32)
            yi = yi.to(device=device, dtype = torch.long)
            scores = model(xi) 
            _, pred = scores.max(dim=1) 
            num_correct += (pred == yi.squeeze()).sum() 
            num_total += pred.size(0)

            return float(num_correct)/num_total   

In [ ]:
def train(model, optimiser, mb_size, epochs=100):
    model = model.to(device=device)
    for epoch in range(epochs):
        for (xi, yi) in create_minibatches(x_train_tensor, y_train_tensor, mb_size):
            model.train()
            xi = xi.to(device=device, dtype=torch.float32)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            cost = F.cross_entropy(input= scores, target=yi.squeeze())
            optimiser.zero_grad()
            cost.backward()
            optimiser.step()
    print(f'Epoch: {epoch}, costo: {cost.item()}, accuracy: {accuracy(model, x_val_tensor, y_val_tensor, mb_size)}')


In [ ]:
hidden1=1000
hidden=1000
lr=0.01
epochs=5
mb_size = 4096
training_model=nn.Sequential(nn.Linear(in_features=784, out_features=hidden1),nn.ReLU(),
                    nn.Linear(in_features=hidden1, out_features=hidden),nn.ReLU(),
                    nn.Linear(in_features=hidden, out_features=10))

optimiser=torch.optim.Adam(training_model.parameters(), lr=lr, betas=(0.9, 0.999))
train(training_model, optimiser, mb_size, epochs)

In [ ]:
accuracy(training_model, x_test_tensor,y_test_tensor,mb_size)

In [ ]:
model_path = './modelPytorchMNIST.pth'

In [ ]:
torch.save(
    training_model.state_dict(),
    model_path)

In [ ]:
hidden1 = 1000 
hidden = 1000
modelMNIST = nn.Sequential(nn.Linear(in_features=784, out_features=hidden1), nn.ReLU(),
                       nn.Linear(in_features=hidden1, out_features=hidden), nn.ReLU(),
                       nn.Linear(in_features=hidden, out_features=10))
modelMNIST

In [ ]:
modelMNIST.load_state_dict(torch.load(model_path))

In [ ]:
modelMNIST = modelMNIST.to(device = 'cuda')
modelMNIST.eval()

In [ ]:
def sample_image():
    rnd_idx = np.random.randint(10000)
    image = x_test[rnd_idx].reshape(1, 28, 28)
    plot_number(image)
    return torch.tensor(image).to(device='cuda', dtype=torch.float).view(1, 784)
    

In [ ]:
image = sample_image()
_, pred = modelMNIST(image).max(1)
print(f'The number is: {pred[0]}')